---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [101]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Import train and test sets
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.preprocessing import MinMaxScaler, LabelEncoder
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_auc_score

    pd.set_option('display.max_columns', None)
    train_set = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
    test_set = pd.read_csv('test.csv', encoding = 'ISO-8859-1')
    addresses_set = pd.read_csv('addresses.csv', encoding = 'ISO-8859-1')
    latlons_set = pd.read_csv('latlons.csv', encoding='ISO-8859-1')

    # Clean the train and test set
    addresses_lat_lon = pd.merge(addresses_set, latlons_set, left_on='address', right_on='address')
    train_set = pd.merge(train_set, addresses_lat_lon, left_on='ticket_id', right_on='ticket_id')
    test_set = pd.merge(test_set, addresses_lat_lon, left_on='ticket_id', right_on='ticket_id')
    train_set = train_set[train_set['country'] == 'USA']
    test_set = test_set[test_set['country'] == 'USA']
    drop_cols_train = ['payment_amount', 'payment_date', 'payment_status', 'balance_due', 'collection_status', 'compliance_detail']
    drop_cols_test = ['grafitti_status', 'non_us_str_code', 'violation_street_name', 'violation_zip_code', 
                      'country', 'state', 'city', 'zip_code', 'address', 'mailing_address_str_number', 
                      'mailing_address_str_name', 'discount_amount', 'clean_up_cost', 'hearing_date',
                      'ticket_issued_date', 'violation_street_number', 'agency_name', 'inspector_name', 'violator_name']
    train_set = train_set[~train_set['compliance'].isnull()]
    train_set = train_set.drop(drop_cols_train + drop_cols_test, axis=1)
    test_set = test_set.drop(drop_cols_test, axis=1) 
    train_set = train_set.dropna()
    test_set['lat'] = test_set['lat'].fillna(np.mean(test_set['lat']))
    test_set['lon'] = test_set['lon'].fillna(np.mean(test_set['lon']))

    # Begin training the set
    scaler = MinMaxScaler()
    lbEncode = LabelEncoder()

    X_train = train_set.loc[:, train_set.columns != 'compliance']
    y_train = train_set.loc[:, 'compliance']
    cols_to_encode = ['disposition', 'violation_code', 'violation_description']

    for labs in cols_to_encode:
        X_train[labs] = lbEncode.fit_transform(X_train[labs])
        test_set[labs] = lbEncode.fit_transform(test_set[labs])

    # Use a Logistic Model
    scaler.fit(X_train)
    X_train_scale = scaler.transform(X_train)
    X_test_scale = scaler.transform(test_set)

    parameters = {'penalty': ['l1', 'l2'], 'C': [0.1, 1, 10]}#{'n_estimators':[10, 100], 'max_depth':[None, 10, 20]}
    #rfc = RandomForestClassifier()
    lgR = LogisticRegression()
    #clf = GridSearchCV(rfc, parameters, scoring='roc_auc').fit(X_train_scale, y_train)
    clf = GridSearchCV(lgR, parameters, scoring='roc_auc').fit(X_train_scale, y_train)

    #pd.Series(clf.predict_proba(X_test_scale), index=test_set['ticket_id'])
    print('Best score: ', clf.best_score_)
    return pd.DataFrame(clf.predict_proba(X_test_scale), test_set['ticket_id']).drop(0, axis=1)# Your answer here

In [102]:
blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Best score:  0.753434394758


,1
ticket_id,
284932,1.242313e-01
285362,1.271368e-02
285361,1.469904e-01
285338,1.279972e-01
285346,1.958935e-01
285345,1.282599e-01
285347,1.641138e-01
285342,4.106240e-01
285530,1.898666e-02


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


array([[ 0.81673691,  0.18326309],
       [ 0.98196763,  0.01803237],
       [ 0.78311017,  0.21688983],
       ..., 
       [ 0.76405844,  0.23594156],
       [ 0.76408252,  0.23591748],
       [ 0.71533058,  0.28466942]])

,0
ticket_id,
284932,0.816737
285362,0.981968
285361,0.783110
285338,0.812998
285346,0.718208
285345,0.812644
285347,0.759555
285342,0.367398
285530,0.973233
